In [ ]:
pip install imgaug opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os
import cv2
import imgaug.augmenters as iaa
from imgaug import parameters as iap
import numpy as np
from tqdm import tqdm

# Define the path to your original and new augmented datasets
original_dataset_path = '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/data/Training'
augmented_dataset_path =  '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/data/Training_augmented'

# Ensure the new directory structure
os.makedirs(augmented_dataset_path, exist_ok=True)

# Define augmentation techniques
augmenters = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.Affine(rotate=(-20, 20))),   # Rotate images
    iaa.Sometimes(0.5, iaa.Affine(scale=(0.8, 1.2))),  # Zoom in/out
    iaa.Sometimes(0.5, iaa.AdditiveGaussianNoise(scale=0.05*255)),  # Add noise
])

# Count the number of files in each class
class_counts = {
    '0Normal': 514,
    '1Doubtful': 477,
    '2Mild': 232,
    '3Moderate': 221,
    '4Severe': 206
}

# Find the maximum number of files in any class
max_files = max(class_counts.values())

# Augment images for each class to match the maximum number of files
for class_name, count in class_counts.items():
    class_path = os.path.join(original_dataset_path, class_name)
    class_files = os.listdir(class_path)
    augmented_class_path = os.path.join(augmented_dataset_path, class_name)
    os.makedirs(augmented_class_path, exist_ok=True)

    # Copy existing images to the new directory
    for file_name in class_files:
        img_path = os.path.join(class_path, file_name)
        img = cv2.imread(img_path)
        cv2.imwrite(os.path.join(augmented_class_path, file_name), img)

    # Calculate the number of augmentations needed
    augment_needed = max_files - count
    for i in tqdm(range(augment_needed), desc=f"Augmenting {class_name}"):
        img_idx = i % count
        img_path = os.path.join(class_path, class_files[img_idx])
        img = cv2.imread(img_path)
        aug_img = augmenters(image=img)
        aug_file_name = f"{os.path.splitext(class_files[img_idx])[0]}_aug_{i}.png"
        cv2.imwrite(os.path.join(augmented_class_path, aug_file_name), aug_img)

print("Data augmentation completed and class sizes equalized.")
